## The Gold Layer: Fraud Scoring
This notebook transforms enriched telemetry into actionable intelligence.

## Defining Fraud Business Rules


In [0]:
from pyspark.sql.functions import when, col# Load the Silver Table

silver_df = spark.readStream.table("fraud_sentinel_catalog.detection_service.silver_transactions")

# Apply scoring logic based on behavioral patterns
# Score > 50 will be considered "High Risk"
gold_scored = (silver_df.withColumn("fraud_score",
                                    when(col("dist_from_home_km") > 500, 40).otherwise(0) + # Large distance jump
                                    when(col("amount") > col("avg_spend") * 5, 30).otherwise(0) + # Massive spend spike
                                    when(col("recipient_status") == "NEW_RECIPIENT", 30).otherwise(0) # Unrecognized recipient
                                    ))

# Filter for the final Fraud Alerts table
high_risk_alerts = gold_scored.filter(col("fraud_score") >= 70)

## Writting to Gold Tables
We will write two outputs: a Summary Table for BI dashboards and an Alert Table for real-time response.


In [0]:

# 1. Configuration for both streams
CHECKPOINT_BASE = "abfss://fraud-sentinel@giftmapote2ete.dfs.core.windows.net/_checkpoints"
ALERTS_PATH = f"{CHECKPOINT_BASE}/gold_alerts"
SUMMARY_PATH = f"{CHECKPOINT_BASE}/gold_summary"

# 2. Start the High-Risk Alerts Stream
print("Starting Write: Gold Fraud Alerts...")
(high_risk_alerts.writeStream
    .format("delta")
    .option("checkpointLocation", ALERTS_PATH)
    .trigger(processingTime='10 seconds') # Reminder: Switch to ProcessingTime for E2E
    .toTable("fraud_sentinel_catalog.detection_service.gold_fraud_alerts"))

# 3. Start the Full Summary Stream
print("Starting Write: Gold Transaction Summary...")
(gold_scored.writeStream
    .format("delta")
    .option("checkpointLocation", SUMMARY_PATH)
    .trigger(processingTime='10 seconds') # Reminder: Switch to ProcessingTime for E2E
    .toTable("fraud_sentinel_catalog.detection_service.gold_transaction_summary"))

print("Success: Both Gold tables have been updated.")

## Checking the data

In [0]:
%sql

select count(*) from fraud_sentinel_catalog.detection_service.gold_fraud_alerts;

## SQL DASHBOARDS

## Real-Time Risk Distribution

In [0]:
%sql
-- SELECT 
--     CASE 
--         WHEN fraud_score >= 70 THEN 'High Risk (Urgent)'
--         WHEN fraud_score >= 40 THEN 'Medium Risk (Monitor)'
--         ELSE 'Low Risk (Safe)' 
--     END AS risk_level,
--     count(*) as transaction_count
-- FROM fraud_sentinel_catalog.detection_service.gold_transaction_summary
-- GROUP BY 1
-- ORDER BY risk_level DESC

Databricks visualization. Run in Databricks to view.

## The Impossible Travel heatmap
This is where we prove the Haversine logic worked. We want to see if the high-risk transactions are indeed happening far from home.

In [0]:
%sql
-- SELECT 
--     dist_from_home_km, 
--     amount, 
--     fraud_score,
--     customer_id
-- FROM fraud_sentinel_catalog.detection_service.gold_transaction_summary
-- WHERE dist_from_home_km > 0
-- ORDER BY dist_from_home_km DESC
-- LIMIT 100

Databricks visualization. Run in Databricks to view.

## TOP 10 AR RISK Customers

In [0]:
%sql
-- SELECT 
--     customer_id, 
--     count(*) as alert_count, 
--     max(dist_from_home_km) as max_distance,
--     sum(amount) as total_flagged_value
-- FROM fraud_sentinel_catalog.detection_service.gold_fraud_alerts
-- GROUP BY customer_id
-- ORDER BY alert_count DESC, total_flagged_value DESC
-- LIMIT 10

Databricks visualization. Run in Databricks to view.